In [13]:
import os
import xarray as xr
import numpy as np
from tqdm import tqdm
import torch


# ====== ⚙️ 1. 설정 (Settings) ⚙️ =======================================================
# --- 데이터 관련 설정 ---
days = 100                                  # 처리할 전체 데이터 일수 (일)
gap_min = 5                                 # 데이터 간격 (분)
delta_min = 30                              # 몇 분 뒤를 예측할 것인지
start_idx = 576000                          # 시작 타임 인덱스 (40일 지점)

# --- 채널 관련 설정 ---
CHANNEL_LIST = [
    'tcwv', 't_lowest', 'u_lowest', 'v_lowest',
    't850', 'z850', 'u850', 'v850',
    't500', 'z500', 'u500', 'v500'
]
NUM_CHANNELS = len(CHANNEL_LIST)

# --- 경로 설정 ---
NC_2D_DIR = "/scratch/x3108a06/nc_data/OUT_2D_nc/"
NC_3D_DIR = "/scratch/x3108a06/nc_data/OUT_3D_nc/"

var_name = '12ch'
base_dir = "/scratch/x3108a06/input_data/hybrid/"
# 최종 .pt 파일을 저장할 디렉토리
output_dir = f'{base_dir}{var_name}_{gap_min}m'
stack_dir = f'{base_dir}{var_name}_d{delta_min}m_{gap_min}m'
# 통계 파일이 저장된 디렉토리 (위 스크립트와 동일)
stats_dir = output_dir

# --- 물리 상수 ---
G_CONST = 9.80665  # 표준 중력가속도 (m/s^2)
# ======================================================================================

In [6]:

# ====================================
# 평균과 표준편차를 계산하여 파일로 저장합니다.
# ====================================


def get_12_channels_from_nc_files(timestep_index: int) -> np.ndarray:
    """
    주어진 타임스텝의 2D와 3D NetCDF 파일에서 12개 채널 데이터를 추출/계산하는 함수.
    """
    file_2d_path = os.path.join(NC_2D_DIR, f"rcemip_small_awing_96x96x74_1km_6s_300K_48_{timestep_index:010d}.2Dcom_1.nc")
    file_3d_path = os.path.join(NC_3D_DIR, f"rcemip_small_awing_96x96x74_1km_6s_300K_48_{timestep_index:010d}.nc")

    if not (os.path.exists(file_2d_path) and os.path.exists(file_3d_path)):
        return None

    with xr.open_dataset(file_2d_path) as ds_2d, xr.open_dataset(file_3d_path) as ds_3d:
        tcwv = ds_2d['PW'].isel(time=0).values
        z = ds_3d['z'].values
        p_hpa = ds_3d['p'].values
        pp = ds_3d['PP'].isel(time=0).values
        tabs = ds_3d['TABS'].isel(time=0).values
        u = ds_3d['U'].isel(time=0).values
        v = ds_3d['V'].isel(time=0).values

        t_lowest = tabs[0, :, :]
        u_lowest = u[0, :, :]
        v_lowest = v[0, :, :]

        geopotential = G_CONST * z
        full_pressure_pa = (p_hpa * 100)[:, np.newaxis, np.newaxis] + pp
        
        pressure_levels_hpa = [850.0, 500.0]
        interpolated_channels = []
        ny, nx = tcwv.shape

        for p_level_hpa in pressure_levels_hpa:
            target_pressure_pa = p_level_hpa * 100.0
            t_interp, z_interp, u_interp, v_interp = (np.zeros((ny, nx)) for _ in range(4))

            for j in range(ny):
                for i in range(nx):
                    p_profile = full_pressure_pa[:, j, i]
                    p_rev, t_rev, u_rev, v_rev, gp_rev = (
                        p_profile[::-1], tabs[:, j, i][::-1], u[:, j, i][::-1],
                        v[:, j, i][::-1], geopotential[::-1]
                    )
                    t_interp[j, i] = np.interp(target_pressure_pa, p_rev, t_rev)
                    z_interp[j, i] = np.interp(target_pressure_pa, p_rev, gp_rev)
                    u_interp[j, i] = np.interp(target_pressure_pa, p_rev, u_rev)
                    v_interp[j, i] = np.interp(target_pressure_pa, p_rev, v_rev)
            
            interpolated_channels.extend([t_interp, z_interp, u_interp, v_interp])

    all_channels = [tcwv, t_lowest, u_lowest, v_lowest] + interpolated_channels
    return np.stack(all_channels, axis=0)


def calculate_and_save_stats():
    """메인 실행 함수: 평균 및 표준편차를 계산하고 저장."""
    print("===== 평균 및 표준편차 계산 시작 =====")
    os.makedirs(stats_dir, exist_ok=True)
    
    mean_array = np.zeros((NUM_CHANNELS,))
    mean2_array = np.zeros((NUM_CHANNELS,))
    count = 0
    
    # 훈련 데이터의 70%를 사용하여 통계치 계산
    loop_end_idx_train = start_idx + (days * 24 * 60 * 10) * 7 // 10
    # 평균 계산 시에는 모든 데이터를 다 볼 필요가 없으므로 간격을 넓게 설정
    for i in tqdm(range(start_idx, loop_end_idx_train + 1, gap_min * 100), desc="Calculating Mean/Std"):
        data_np = get_12_channels_from_nc_files(i)
        if data_np is None: continue

        mean_array += data_np.mean(axis=(1, 2))
        mean2_array += (data_np**2).mean(axis=(1, 2))
        count += 1
    
    if count == 0:
        print("오류: 평균/표준편차를 계산할 데이터를 찾지 못했습니다. 파일 경로와 인덱스를 확인하세요.")
        return

    mean = mean_array / count
    mean2 = mean2_array / count
    std = np.sqrt(mean2 - mean**2)

    # 계산된 통계치 저장
    mean_std_filename = f"mean_std_{days}d_{gap_min}m.npy"
    mean_std_path = os.path.join(stats_dir, mean_std_filename)
    np.save(mean_std_path, np.array([mean, std]))
    
    print(f"\n===== 계산 완료! =====")
    print(f"평균/표준편차 저장 완료: {mean_std_path}")
    print("Mean:", mean)
    print("Std:", std)



if __name__ == '__main__':
    calculate_and_save_stats()

In [7]:

# ======================================
# 데이터를 정규화, 패딩하고 .pt 파일로 저장합니다.
# ======================================


def cyclic_pad_2d(x: torch.Tensor, pad: int) -> torch.Tensor:
    """주기적 경계 조건으로 2D 텐서를 패딩하는 함수."""
    x = torch.cat([x[..., -pad:, :], x, x[..., :pad, :]], dim=-2)
    x = torch.cat([x[..., :, -pad:], x, x[..., :, :pad]], dim=-1)
    return x


def normalize_and_save_data():
    """메인 실행 함수: 데이터를 정규화, 패딩하고 저장."""
    print("===== 데이터 정규화 및 저장 시작 =====")
    os.makedirs(output_dir, exist_ok=True)
    
    # --- 1. 저장된 평균/표준편차 불러오기 ---
    mean_std_filename = f"mean_std_{days}d_{gap_min}m.npy"
    mean_std_path = os.path.join(stats_dir, mean_std_filename)
    
    if not os.path.exists(mean_std_path):
        print(f"오류: 통계 파일({mean_std_path})을 찾을 수 없습니다.")
        print("먼저 '평균 및 표준편차 계산 스크립트'를 실행해주세요.")
        return
        
    mean, std = np.load(mean_std_path)
    print(f"통계 파일 불러오기 완료: {mean_std_path}")
    
    # Torch 텐서로 변환하여 GPU 사용 준비
    mean_tensor = torch.from_numpy(mean).float().view(NUM_CHANNELS, 1, 1)
    std_tensor = torch.from_numpy(std).float().view(NUM_CHANNELS, 1, 1)

    # --- 2. 전체 데이터를 순회하며 정규화, 패딩 및 저장 ---
    loop_end_idx_full = start_idx + days * 24 * 60 * 10
    # 모든 데이터를 처리해야 하므로 간격을 촘촘하게 설정
    for i in tqdm(range(start_idx, loop_end_idx_full + 1, gap_min * 10), desc="Normalizing and Saving"):
        if os.path.exists(os.path.join(output_dir, f"{i:010d}.pt")):
            continue  # 이미 있으면 다음 파일로 바로 넘어감

        data_np = get_12_channels_from_nc_files(i)
        if data_np is None: continue
        
        data_tensor = torch.from_numpy(data_np).float()
        
        # 정규화 (Standardization)
        normalized_tensor = (data_tensor - mean_tensor) / std_tensor
        
        # 주기적 패딩 적용
        padded_tensor = cyclic_pad_2d(normalized_tensor, pad=16)

        torch.save(padded_tensor, os.path.join(output_dir, f"{i:010d}.pt"))

    print("\n===== 모든 전처리 작업 완료! =====")


if __name__ == '__main__':
    normalize_and_save_data()


===== 데이터 정규화 및 저장 시작 =====
통계 파일 불러오기 완료: /scratch/x3108a06/input_data/hybrid/12ch_5m/mean_std_100d_5m.npy


Normalizing and Saving: 100%|██████████| 28801/28801 [00:02<00:00, 10296.13it/s]


===== 모든 전처리 작업 완료! =====


In [15]:
# =======================
#      (x, y) stack
# =======================

# x, y를 첫번째 차원으로 쌓아서 (2, C, H, W) 모양의 텐서를 만드는 코드

os.makedirs(stack_dir, exist_ok=True)

# 파일 리스트 정렬
filelist = sorted([f for f in os.listdir(output_dir) if f.endswith('.pt')])
N = len(filelist)

# 몇번째 뒤와 x, y쌍으로 묶을 것인지
shift = delta_min // gap_min

for idx, fname in tqdm(enumerate(filelist)):  # type: int, str
    # 최종 결과 파일의 경로를 먼저 확인합니다.
    output_fpath = os.path.join(stack_dir, fname)

    # 만약 결과 파일이 이미 존재한다면, 이번 순서는 건너뜁니다.
    if os.path.exists(output_fpath):
        continue  # 다음 파일로 바로 넘어감

    fpath = os.path.join(output_dir, fname)
    data = torch.load(fpath, map_location='cpu')
    if not isinstance(data, torch.Tensor):
        print(f"{fname}: Not a tensor, skipped.")
        continue

    # 현재 파일
    # padded_0 = cyclic_pad_2d(data, 16)

    # shift만큼 뒤 파일(없으면 마지막 파일 사용)
    shifted_idx = min(idx + shift, N - 1)
    shifted_fname = filelist[shifted_idx]
    shifted_fpath = os.path.join(output_dir, shifted_fname)
    shifted_data = torch.load(shifted_fpath, map_location='cpu')
    
    # padded_1 = cyclic_pad_2d(shifted_data, 16)

    # (2, 128, 128) 저장
    out = torch.stack([data, shifted_data], dim=0)
    torch.save(out, os.path.join(stack_dir, fname))


20034it [35:51,  9.31it/s]


In [16]:
# =======================
#    train test split
# =======================

import shutil
from math import floor


# 앞에서부터 70%를 train으로, 15%를 valid로, 15%를 test 셋으로 분리하여 저장하는 코드

def split_and_copy(src_dir, dst_base, prefix='', total_limit=None):
    os.makedirs(dst_base, exist_ok=True)
    files = sorted([f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))])
    if total_limit:
        files = files[:total_limit]
    n = len(files)
    n_train = floor(n * 0.7)
    n_valid = floor(n * 0.15)
    n_test = n - n_train - n_valid

    splits = [
        ('train', n_train),
        ('valid', n_valid),
        ('test', n_test),
    ]

    print(f"[{prefix}] Total: {n} -> train: {n_train}, valid: {n_valid}, test: {n_test}")

    idx = 0
    for split, count in splits:
        dst_dir = os.path.join(dst_base, prefix, split)
        os.makedirs(dst_dir, exist_ok=True)
        for i in tqdm(range(count), desc=split):
            src_path = os.path.join(src_dir, files[idx])
            dst_path = os.path.join(dst_dir, files[idx])
            idx += 1
            if os.path.exists(dst_path):
                continue  # 다음 파일로 바로 넘어감
            shutil.copy2(src_path, dst_path)


# ============== settings ====================
split_and_copy(
    src_dir=stack_dir,
    dst_base=base_dir,
    prefix=f'{var_name}_d{delta_min}m_{gap_min}m',
    total_limit=None
)
# ============================================

[12ch_d30m_5m] Total: 20034 -> train: 14023, valid: 3005, test: 3006


test: 100%|██████████| 3006/3006 [02:07<00:00, 23.66it/s]
